# MPContribs

## Walkthrough

- start with a [materials detail page](https://next-gen.materialsproject.org/materials/mp-2715) on MP with user contributions
- navigate to https://mpcontribs.org and explore
- apply for project on https://workshop-contribs.materialsproject.org/contribute (wait for approval by admins)

In [ ]:
name = "your-project-name"
apikey = "your-api-key"  # profile.materialsproject.org

## Contribute data on Refractive Index

We'll prepare [refractive index data](https://refractiveindex.info) for contribution to https://workshop-contribs.materialsproject.org using the [mpcontribs-client](https://pypi.org/project/mpcontribs-client/).

In [ ]:
from mpcontribs.client import Client
from mp_api.matproj import MPRester
from refractivesqlite import dboperations as DB
from pandas import DataFrame

### Explore and extract refractive index data

https://github.com/HugoGuillen/refractiveindex.info-sqlite/blob/master/Tutorial.ipynb

In [ ]:
db = DB.Database("refractive.db")
#db.create_database_from_url()

In [ ]:
db.search_pages("Au", exact=True)

In [ ]:
materials = db.search_custom(
    'select * from pages where book="Au" and hasrefractive=1 and hasextinction=1'
)

### Prepare a single contribution for testing

Note that a contribution to a specific MP material contains 4 optional components:

- simple (potentially nested) "key-value" `data`
- `tables` as Pandas DataFrame objects (think spreadsheets and csv files)
- `structures` as Pymatgen Structure objects (think CIF, POSCAR, ...)
- `attachments` (think gzipped text files, PNG/JPG..)

Example for a single contribution dictionary:

```
{
    "project": "sandbox",
    "identifier": "mp-4",
    "data": {
        "a": "3 eV",
        "b": {"c": "hello", "d": 3},
        "d.e.f": "nest via dot-notation"
    },
    "structures": [<pymatgen Structure>, ...],
    "tables": [<pandas DataFrame>, ...],
    "attachments": [<pathlib.Path>, <mpcontribs.client.Attachment>, ...]
}
```

In [ ]:
page_id = materials[0][0]
mat = db.get_material(page_id)

In [ ]:
mpr = MPRester(apikey)

In [ ]:
def get_contrib(mat):
    info = mat.get_page_info()
    formula = info["book"]
    mpid = mpr.get_materials_ids(formula)[0]

    rmin, rmax = info['rangeMin']*1000, info['rangeMax']*1000
    mid = (rmin + rmax) / 2
    n = mat.get_refractiveindex(mid)
    k = mat.get_extinctioncoefficient(mid)
    
    x = "wavelength λ [μm]"
    refrac = DataFrame(mat.get_complete_refractive(), columns=[x, "n"])
    refrac.set_index(x, inplace=True)
    extinct = DataFrame(mat.get_complete_extinction(), columns=[x, "k"])
    extinct.set_index(x, inplace=True)

    df = refrac.join(extinct["k"])
    df.attrs["title"] = f"Complex refractive index (n+ik) for {formula}"
    df.attrs["labels"] = {
        "value": "n, k",  # y-axis label
        "variable": "Re/Im" # legend name (= df.columns.name)
    }
    df.plot(**df.attrs)#.show()
    df.attrs["name"] = "n,k(λ)"
    return {
        "project": name,
        "identifier": str(mpid),
        "data": {
            "n": float(n),
            "k": float(k),
            "range.min": f"{rmin} nm",
            "range.mid": f"{mid} nm",
            "range.max": f"{rmax} nm",
            "points": info["points"],
            "page": info["page"]
        },
        "tables": [df]
    }

In [ ]:
contrib = get_contrib(mat)

### Retrieve and update project info

Let's add the URL for the DB and also set `unique_identifiers` to `False`. This flag indicates that this project can contain multiple contributions (rows in the landing page's overview table) for the same MP material (`mp-id`). We also want to include descriptions of the `data` columns in the project's `other` field.

In [ ]:
client = Client(
    host="workshop-contribs-api.materialsproject.org",
    apikey=apikey
)

In [ ]:
db.check_url_version()

In [ ]:
update = {
    'unique_identifiers': False,
    'references': [
        {'label': 'website', 'url': 'https://refractiveindex.info'},
        {'label': 'source', 'url': "https://refractiveindex.info/download/database/rii-database-2019-02-11.zip"}
    ],
    "other": { # describe the root fields here to automatically include tooltips on MP
        "n": "real part of complex refractive index",
        "k": "imaginary part of complex refractive index",
        "range": "wavelength range for n,k in nm",
        "points": "number of λ points in range",
        "page": "reference to data source/publication"
    }
}
# could also update authors, title, long_title, description

In [ ]:
client.projects.update_entry(pk=name, project=update).result()

Try searching for `refractive` in MPContribs browse page now

### Initialize `data` columns

In [ ]:
client.init_columns(name, {
    "n": "", # dimensionless
    "k": "",
    "range.min": "nm",
    "range.mid": "nm",
    "range.max": "nm",
    "points": "",
    "page": None # text    
})

### Submit test contribution

Simply provide your list of contributions as argument to the client's `submit_contributions` function to prepare and upload them to MPContribs. Duplicate checking will happen automatically if `unique_identifers` is set to `True` for the project (the default). If successful, the client will return the number of contributions submitted.

In [ ]:
client.submit_contributions([contrib])

Your first contribution should now show up in your project on https://workshop-contribs.materialsproject.org :)

### Prepare and upload all contributions

In [ ]:
contributions = []

for material in materials:
    page_id = material[0]
    mat = db.get_material(page_id)
    contrib = get_contrib(mat)
    contributions.append(contrib)

In [ ]:
contributions[10]

In [ ]:
client.delete_contributions(name)
client.submit_contributions(contributions, ignore_dupes=True)

### Publish contributions

In [ ]:
client.make_public(name)

### Retrieve and query contributions

In [ ]:
all_ids = client.get_all_ids(
    {"project": name},
    include=["tables"],
    data_id_fields={name: "page"},
    fmt="map"
)

Grab the table ID and retrieve it as Pandas DataFrame. Show a graph.

In [ ]:
tid = all_ids[name]["mp-81"]["Johnson"]["tables"]["n,k(λ)"]["id"]
client.get_table(tid).display()

Finally, let's build up a more complicated query to reduce the list of contributions to the ones we might be interested in.

In [ ]:
query = {
    "project": name,
    "formula__contains": "Au",
    #"identifier__in": []

    "data__n__value__lt": 200,
    "data__k__value__gte": 7,

    "_sort": "-data__range__mid__value",
    "_fields": [
        "id", "identifier", "formula",
        "data.range.mid.value",
        "data.n.value",
        "data.k.value"
    ]
}

print(client.get_totals(query))
client.contributions.get_entries(**query).result()

## Exercises


- pick a different book/element and upload more refraction index data to your project
- make yourself familiar with datasets available on MPContribs
- use the `Search` tab to find interesting contributions and their MP detail page(s)
- query the `carrier_transport` or another dataset of your choice
- retrieve another table, display and plot it interactively
- delete the refraction index data and start adding your own :)